# Import Libraries

In [1]:
import json
from datetime import datetime
from IPython.display import clear_output 
import numpy as np
import pandas as pd
import pytz
from time import gmtime, strftime

!pip install yahoo_fin --upgrade --no-cache-dir
clear_output()

In [2]:
now= datetime.now()
date_time = now.strftime("%Y-%m-%d %H:%M:%S.%f")
tzs = 'GMT+11'
tza = 'GMT-5'
print(date_time)
print(tzs)
print(tza)


2023-02-09 07:00:52.787880
GMT+11
GMT-5


# Tool for visualizing json format
https://jsoncrack.com/editor

# Data model template

In [3]:
# Our data model
jsonfile_example = {"data_source":"datasource_X", "dataset_type_id": "http://bucket-name.s3-website-Region.amazonaws.com", 
                        "time_object":{"timestamp":date_time, "timezone":tzs},
             "events":[{"time_object":{"timestamp":'2019-07-21 13:04:40.3401012', "duration":1, "timezone":tzs},"event_type":'sensor reading',   
                          "attribute":{"attr1":36.0, "attr2":"abc", "attr3":False}},
                       {"time_object":{"timestamp":'2019-07-22 13:04:40.301022', "duration":1, "timezone":tzs},"event_type":'sensor reading',   
                          "attribute":{"attr1":37.0, "attr2":"bcd", "attr3":True}},
                       ],  
             }
# Write pretty print JSON data to file
with open("jsonfile_example.json", "w") as write_file:
    json.dump(jsonfile_example, write_file, indent=4)

jsonfile_example

{'data_source': 'datasource_X',
 'dataset_type_id': 'http://bucket-name.s3-website-Region.amazonaws.com',
 'time_object': {'timestamp': '2023-02-09 07:00:52.787880',
  'timezone': 'GMT+11'},
 'events': [{'time_object': {'timestamp': '2019-07-21 13:04:40.3401012',
    'duration': 1,
    'timezone': 'GMT+11'},
   'event_type': 'sensor reading',
   'attribute': {'attr1': 36.0, 'attr2': 'abc', 'attr3': False}},
  {'time_object': {'timestamp': '2019-07-22 13:04:40.301022',
    'duration': 1,
    'timezone': 'GMT+11'},
   'event_type': 'sensor reading',
   'attribute': {'attr1': 37.0, 'attr2': 'bcd', 'attr3': True}}]}

# Example 1.Yahoo finance 

## a. yahoo finance quote data

In [4]:
# get raw data

import yahoo_fin.stock_info as si
quote_table = si.get_quote_table("aapl")
quote_table


/home/codespace/.python/current/lib/python3.10/site-packages/yahoo_fin/stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
/home/codespace/.python/current/lib/python3.10/site-packages/yahoo_fin/stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


{'1y Target Est': 169.17,
 '52 Week Range': '124.17 - 179.61',
 'Ask': '0.00 x 900',
 'Avg. Volume': 77873885.0,
 'Beta (5Y Monthly)': 1.28,
 'Bid': '0.00 x 800',
 "Day's Range": '151.17 - 154.58',
 'EPS (TTM)': 5.89,
 'Earnings Date': 'Apr 26, 2023 - May 01, 2023',
 'Ex-Dividend Date': 'Feb 10, 2023',
 'Forward Dividend & Yield': '0.92 (0.59%)',
 'Market Cap': '2.404T',
 'Open': 153.88,
 'PE Ratio (TTM)': 25.79,
 'Previous Close': 154.65,
 'Quote Price': 151.9199981689453,
 'Volume': 64120079.0}

In [5]:
# Our data model
jsonfile_quote = {"data_source":"yahoo_finance", "dataset_type_id": "http://bucket-name.s3-website-Region.amazonaws.com",
                  "time_object":{"timestamp":date_time, "timezone":tzs},
             "events":[{"time_object":{"timestamp":'2019-07-21 13:04:40.3401012', "duration":1, "timezone":tzs},
                        "event_type":'stock quote',   
                        "attribute":quote_table, }],  
             }
# Write pretty print JSON data to file
with open("jsonfile_quote.json", "w") as write_file:
    json.dump(jsonfile_quote, write_file, indent=4)

jsonfile_quote

{'data_source': 'yahoo_finance',
 'dataset_type_id': 'http://bucket-name.s3-website-Region.amazonaws.com',
 'time_object': {'timestamp': '2023-02-09 07:00:52.787880',
  'timezone': 'GMT+11'},
 'events': [{'time_object': {'timestamp': '2019-07-21 13:04:40.3401012',
    'duration': 1,
    'timezone': 'GMT+11'},
   'event_type': 'stock quote',
   'attribute': {'1y Target Est': 169.17,
    '52 Week Range': '124.17 - 179.61',
    'Ask': '0.00 x 900',
    'Avg. Volume': 77873885.0,
    'Beta (5Y Monthly)': 1.28,
    'Bid': '0.00 x 800',
    "Day's Range": '151.17 - 154.58',
    'EPS (TTM)': 5.89,
    'Earnings Date': 'Apr 26, 2023 - May 01, 2023',
    'Ex-Dividend Date': 'Feb 10, 2023',
    'Forward Dividend & Yield': '0.92 (0.59%)',
    'Market Cap': '2.404T',
    'Open': 153.88,
    'PE Ratio (TTM)': 25.79,
    'Previous Close': 154.65,
    'Quote Price': 151.9199981689453,
    'Volume': 64120079.0}}]}

## b. yahoo finance OHLC data

In [6]:
# get raw data
import yahoo_fin.stock_info as si
ohlc = si.get_data("aapl")
ohlc['date'] = ohlc.index
ohlc['date'] = pd.to_datetime(ohlc['date'],format='%Y-%m-%d %00:00:00.00 %Z' ).astype(str)
ohlc.head()

,open,high,low,close,adjclose,volume,ticker,date
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099874,469033600,AAPL,1980-12-12
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094663,175884800,AAPL,1980-12-15
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087715,105728000,AAPL,1980-12-16
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089886,86441600,AAPL,1980-12-17
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092492,73449600,AAPL,1980-12-18


In [7]:
# transform in to json
import ast
ohlc_json = ohlc.to_json(orient="records")
ohlc_json = ast.literal_eval(ohlc_json)
ohlc_json = ohlc_json[0:3]
ohlc_json

[{'open': 0.1283479929,
  'high': 0.1289059967,
  'low': 0.1283479929,
  'close': 0.1283479929,
  'adjclose': 0.0998737887,
  'volume': 469033600,
  'ticker': 'AAPL',
  'date': '1980-12-12'},
 {'open': 0.1222100034,
  'high': 0.1222100034,
  'low': 0.1216519997,
  'close': 0.1216519997,
  'adjclose': 0.0946633071,
  'volume': 175884800,
  'ticker': 'AAPL',
  'date': '1980-12-15'},
 {'open': 0.1132809967,
  'high': 0.1132809967,
  'low': 0.1127230003,
  'close': 0.1127230003,
  'adjclose': 0.0877152458,
  'volume': 105728000,
  'ticker': 'AAPL',
  'date': '1980-12-16'}]

In [8]:
# Our data model
jsonfile_ohlc = {"data_source":"yahoo_finance", "dataset_type_id": "http://bucket-name.s3-website-Region.amazonaws.com", 
                 "time_object":{"timestamp":date_time, "timezone":tzs},
                  "events":[]}

for i in range(len(ohlc_json)):
  datetime_object = pd.to_datetime(ohlc_json[i]['date'], format='%Y-%m-%d %H:%M:%S.%f' )
  datetime_object= datetime_object.strftime('%Y-%m-%d %H:%M:%S.%f')
  jsonfile_ohlc['events'].append({"time_object":{"timestamp":datetime_object, "duration":24*60*60, "timezone":tzs},
                                  "event_type":'stock ohlc',   "attribute":ohlc_json[i] })

# Write pretty print JSON data to file
with open("jsonfile_ohlc.json", "w") as write_file:
    json.dump(jsonfile_ohlc, write_file, indent=4)
    
jsonfile_ohlc

{'data_source': 'yahoo_finance',
 'dataset_type_id': 'http://bucket-name.s3-website-Region.amazonaws.com',
 'time_object': {'timestamp': '2023-02-09 07:00:52.787880',
  'timezone': 'GMT+11'},
 'events': [{'time_object': {'timestamp': '1980-12-12 00:00:00.000000',
    'duration': 86400,
    'timezone': 'GMT+11'},
   'event_type': 'stock ohlc',
   'attribute': {'open': 0.1283479929,
    'high': 0.1289059967,
    'low': 0.1283479929,
    'close': 0.1283479929,
    'adjclose': 0.0998737887,
    'volume': 469033600,
    'ticker': 'AAPL',
    'date': '1980-12-12'}},
  {'time_object': {'timestamp': '1980-12-15 00:00:00.000000',
    'duration': 86400,
    'timezone': 'GMT+11'},
   'event_type': 'stock ohlc',
   'attribute': {'open': 0.1222100034,
    'high': 0.1222100034,
    'low': 0.1216519997,
    'close': 0.1216519997,
    'adjclose': 0.0946633071,
    'volume': 175884800,
    'ticker': 'AAPL',
    'date': '1980-12-15'}},
  {'time_object': {'timestamp': '1980-12-16 00:00:00.000000',
    'd

## c. exmaple yahoo finance news data

In [9]:
# get raw data
from yahoo_fin import news

news_raw = news.get_yf_rss("aapl")
news_raw[0] # only use 1 news


{'summary': '(Bloomberg) -- Apple Inc. is hiring its first chief people officer and shifting human resources duties from its head of retail, overhauling the way the tech giant hires and supports employees.Most Read from BloombergMeta Asks Many Managers to Get Back to Making Things or LeaveGeorge Santos Gets Into Fight With Mitt Romney at State of the Union DebutDeSantis Chides Trump as Republicans’ 2024 Presidential Race Heats UpChinese Balloon Was Part of Years-Long Spying Program, US SaysRussia Will Fail t',
 'summary_detail': {'type': 'text/html',
  'language': None,
  'base': 'https://feeds.finance.yahoo.com/rss/2.0/headline?s=aapl&region=US&lang=en-US',
  'value': '(Bloomberg) -- Apple Inc. is hiring its first chief people officer and shifting human resources duties from its head of retail, overhauling the way the tech giant hires and supports employees.Most Read from BloombergMeta Asks Many Managers to Get Back to Making Things or LeaveGeorge Santos Gets Into Fight With Mitt Romn

In [11]:
# Our data model
jsonfile_news1 = {"data_source":"yahoo_finance", "dataset_type_id": "http://bucket-name.s3-website-Region.amazonaws.com",
                  "time_object":{"timestamp":date_time, "timezone":tzs},
                  "events":[]  
                 }

print("======================news 1======================")

for i in range(len(news_raw)):
  datetime_str = news_raw[i]['published'][5:]
  datetime_object = datetime.strptime(datetime_str, '%d %b %Y %H:%M:%S +0000')
  datetime_object = pd.to_datetime(datetime_object, format='%Y-%m-%d %H:%M:%S.%f' )
  datetime_object= datetime_object.strftime('%Y-%m-%d %H:%M:%S.%f')
  jsonfile_news1["events"].append({"time_object":{"timestamp":datetime_object, "duration":24*60*60, "timezone":tzs},
                                   "event_type":'stock news',   
                                   "attribute":news_raw[i] } )
  if i == 1:
    break

# Write pretty print JSON data to file
with open("jsonfile_news.json", "w") as write_file:
    json.dump(jsonfile_news1, write_file, indent=4)

jsonfile_news1


======================news 1======================


{'data_source': 'yahoo_finance',
 'dataset_type_id': 'http://bucket-name.s3-website-Region.amazonaws.com',
 'time_object': {'timestamp': '2023-02-09 07:00:52.787880',
  'timezone': 'GMT+11'},
 'events': [{'time_object': {'timestamp': '2023-02-08 22:00:19.000000',
    'duration': 86400,
    'timezone': 'GMT+11'},
   'event_type': 'stock news',
   'attribute': {'summary': '(Bloomberg) -- Apple Inc. is hiring its first chief people officer and shifting human resources duties from its head of retail, overhauling the way the tech giant hires and supports employees.Most Read from BloombergMeta Asks Many Managers to Get Back to Making Things or LeaveGeorge Santos Gets Into Fight With Mitt Romney at State of the Union DebutDeSantis Chides Trump as Republicans’ 2024 Presidential Race Heats UpChinese Balloon Was Part of Years-Long Spying Program, US SaysRussia Will Fail t',
    'summary_detail': {'type': 'text/html',
     'language': None,
     'base': 'https://feeds.finance.yahoo.com/rss/2.0/he

Bad pipe message: %s [b'xz\xd3 \x1d\xfc\xbf<FjfD2P\xb6\x8b\x88S \xebu\x87a\x9f\x00;6s\tyf\x96\xe7\x86\xbdv\xc7\xbf\x85\x17\xa1J\xfe\xf0']
Bad pipe message: %s [b'\xa9\xef\xc6\xde\x95\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t', b'\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b"\xb8\xf7\xa5+\xed\xe0\xb4\x80\x8eP^t\xd1o\xe9\x80\x93\x80\x00\x00\xf4\xc00\xc0,\xc0(\xc0$\xc0\x14\xc0\n\x00\xa5\x00\xa3\x00\xa1\x00\x9f\x00k\x00j\x00i\x00h\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00\xa7\x00m\x00:\x00\x89\xc02\xc0.\xc0*\xc0&\xc0\x0f\xc0\x05\x00\x9d\x00=\x005\x00\x84\xc0/\xc0+\xc0'\xc0#\xc0\x13\xc0\t\x00\xa4\x00\xa2\x00\xa0\x00\x9e\x00", b'@\x00?\x00>\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00\x97\x00E\x00D\x0